In [1]:
import tensorflow as tf

In [25]:
import numpy as np
import keras
from keras.layers import Input, Dense, Lambda, InputLayer, concatenate
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
from keras.utils import np_utils

In [5]:
a = tf.constant([[1,1,1],[2,2,2]],dtype=tf.float32)
a

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [2., 2., 2.]], dtype=float32)>

In [6]:
b = 5 - a
b*a

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[4., 4., 4.],
       [6., 6., 6.]], dtype=float32)>

In [7]:
tf.exp(a)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2.7182817, 2.7182817, 2.7182817],
       [7.389056 , 7.389056 , 7.389056 ]], dtype=float32)>

In [8]:
a**2

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [4., 4., 4.]], dtype=float32)>

In [9]:
# trying
X = tf.constant([[1,0,1],[0,0,1]],shape=(2, 3),dtype=tf.float32)
X
X_mean = tf.constant([[0.1,0.5,0.8],[0.1,0.2,0.4]],shape=(2, 3),dtype=tf.float32)
tf.math.multiply(X,tf.math.log(X_mean))
tf.math.multiply(1-X,tf.math.log(X_mean))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.       , -0.6931472, -0.       ],
       [-2.3025851, -1.609438 , -0.       ]], dtype=float32)>

In [11]:
x = tf.constant([[1, 1, 1], [1, 1, 1]])
tf.reduce_sum(x)  # [2, 2, 2] ## [3, 3]


<tf.Tensor: shape=(), dtype=int32, numpy=6>

In [13]:
import tensorflow.compat.v1 as tf

x = tf.placeholder(tf.float32)
y = tf.square(x)
z = tf.add(x, y)

sess = tf.Session()

z0 = sess.run([z], feed_dict={x: 2.})        # 6.0
z1 = sess.run([z], feed_dict={x: 2., y: 2.}) # 4.0

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [16]:
import tensorflow as tf

@tf.function
def compute_z1(x, y):
  return tf.add(x, y)


def compute_z0(x):
       return compute_z1(x, tf.square(x))

z0 = compute_z0(2.)
z1 = compute_z1(2., 2.)

In [15]:
z0

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [23]:
def vlb_binomial(x, x_decoded_mean, t_mean, t_log_var):
    """Returns the value of negative Variational Lower Bound
    
    The inputs are tf.Tensor
        x: (batch_size x number_of_pixels) matrix with one image per row with zeros and ones
        x_decoded_mean: (batch_size x number_of_pixels) mean of the distribution p(x | t), real numbers from 0 to 1
        t_mean: (batch_size x latent_dim) mean vector of the (normal) distribution q(t | x)
        t_log_var: (batch_size x latent_dim) logarithm of the variance vector of the (normal) distribution q(t | x)
    
    Returns:
        A tf.Tensor with one element (averaged across the batch), VLB
    """
    ### YOUR CODE HERE
    # for part log(p(xi | hat(ti),w)) :
    # assume black is 1 and white is 0
    
    
    return tf.reduce_sum(x + x_decoded_mean) + tf.reduce_sum(t_mean+t_log_var)

# 尝试解决 Symbolic Tensor不兼容的问题
理解： 下述代码原先是通过执行一个Sybolic tensor来凑成一个model。与那种定义一整个Model，每步都很清楚的那种，这个形式上比较自由打散。

In [24]:
test_data = np.load('test_data.npz')
loss = vlb_binomial(test_data['x'], test_data['x_decoded_mean'], test_data['t_mean'], test_data['t_log_var'])
loss

<tf.Tensor: shape=(), dtype=float32, numpy=18209.518>

In [48]:

batch_size = 100
original_dim = 784 # Number of pixels in MNIST images.
latent_dim = 3 # d, dimensionality of the latent code t.
intermediate_dim = 256 # Size of the hidden layer.
epochs = 2

x = Input(batch_shape=(batch_size, original_dim))

In [29]:
def create_encoder(input_dim):
    # Encoder network.
    # We instantiate these layers separately so as to reuse them later
    encoder = Sequential(name='encoder')
    encoder.add(InputLayer([input_dim]))
    encoder.add(Dense(intermediate_dim, activation='relu'))
    encoder.add(Dense(2 * latent_dim))
    return encoder

original_dim =784
intermediate_dim  = 2
latent_dim = 2
encoder = create_encoder(original_dim)

In [30]:
encoder

In [67]:
get_t_mean = Lambda(lambda h: h[:, :latent_dim])
get_t_log_var = Lambda(lambda h: h[:, latent_dim:])

h = encoder(x) # 这里使用了 numpy的数据？
t_mean = get_t_mean(h)
t_log_var = get_t_log_var(h)

In [68]:
t_log_var

<tf.Tensor 'lambda_21/strided_slice:0' shape=(100, 1) dtype=float32>

In [69]:
def sampling(args):
    """Returns sample from a distribution N(args[0], diag(args[1]))
    
    The sample should be computed with reparametrization trick.
    
    The inputs are tf.Tensor
        args[0]: (batch_size x latent_dim) mean of the desired distribution
        args[1]: (batch_size x latent_dim) logarithm of the variance vector of the desired distribution
    
    Returns:
        A tf.Tensor of size (batch_size x latent_dim), the samples.
    """
    t_mean, t_log_var = args
    # YOUR CODE HERE
    sample_std = tf.math.exp(t_log_var / 2.0)
    sample_shape = t_mean.shape
    sampled_normal = tf.random.normal(sample_shape)
    return t_mean + sample_std * sampled_normal


t = Lambda(sampling)([t_mean, t_log_var]) 
"""
这里对Lambda的使用方法类似前端JS的 IIFE (immediately Invoked Function Expression)
因此 这一步是evaluation
"""

def create_decoder(input_dim):
    # Decoder network
    # We instantiate these layers separately so as to reuse them later
    decoder = Sequential(name='decoder')
    decoder.add(InputLayer([input_dim]))
    decoder.add(Dense(intermediate_dim, activation='relu'))
    decoder.add(Dense(original_dim, activation='sigmoid'))
    return decoder
decoder = create_decoder(latent_dim)
x_decoded_mean = decoder(t)

In [70]:
t = Lambda(sampling)([t_mean, t_log_var]) 
t

<tf.Tensor 'lambda_23/add:0' shape=(100, 3) dtype=float32>

In [71]:
x_decoded_mean

<tf.Tensor 'decoder/dense_8/Sigmoid:0' shape=(100, 784) dtype=float32>

In [72]:
vae = Model(x, x_decoded_mean)

In [73]:
vae.summary()

Model: "functional_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(100, 784)]         0                                            
__________________________________________________________________________________________________
encoder (Sequential)            (None, 4)            1582        input_7[0][0]                    
__________________________________________________________________________________________________
lambda_20 (Lambda)              (100, 3)             0           encoder[4][0]                    
__________________________________________________________________________________________________
lambda_21 (Lambda)              (100, 1)             0           encoder[4][0]                    
_______________________________________________________________________________________

In [59]:
keras.utils.plot_model(vae, "my_first_model.png")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [64]:
loss = vlb_binomial(x, x_decoded_mean, t_mean, t_log_var)
loss

<tf.Tensor 'AddV2_2:0' shape=() dtype=float32>

In [74]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

vae.compile(optimizer=keras.optimizers.RMSprop(lr=0.001), loss=lambda x, y: loss)

In [76]:
vae.summary()

Model: "functional_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(100, 784)]         0                                            
__________________________________________________________________________________________________
encoder (Sequential)            (None, 4)            1582        input_7[0][0]                    
__________________________________________________________________________________________________
lambda_20 (Lambda)              (100, 3)             0           encoder[4][0]                    
__________________________________________________________________________________________________
lambda_21 (Lambda)              (100, 1)             0           encoder[4][0]                    
_______________________________________________________________________________________

In [78]:
with tf.GradientTape():
    hist = vae.fit(x=x_train, y=x_train,
                   shuffle=True,
                   epochs=epochs,
                   batch_size=batch_size,
                   validation_data=(x_test, x_test),
                   verbose=2)

Epoch 1/3


ValueError: in user code:

    c:\program files\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\program files\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\program files\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\program files\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\program files\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\program files\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step

In [62]:
test_scores = vae.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/2
479/480 [============================>.] - ETA: 0s - loss: 5.9368 - accuracy: 0.1139

InvalidArgumentError:  Incompatible shapes: [64,1] vs. [100,3]
	 [[node functional_2/lambda_18/mul (defined at <ipython-input-54-874927fd1ecf>:18) ]] [Op:__inference_test_function_4992]

Errors may have originated from an input operation.
Input Source operations connected to node functional_2/lambda_18/mul:
 functional_2/lambda_18/Exp (defined at <ipython-input-54-874927fd1ecf>:15)	
 functional_2/lambda_18/random_normal (defined at <ipython-input-54-874927fd1ecf>:17)

Function call stack:
test_function
